### Sentiment Analysis of Instagram posts 
Script copywrite: Aman, Jayedi;
Written: 20210212

#### take a look here#
https://colab.research.google.com/drive/1xyaAMav_gTo_KvpHrO05zWFhmUaILfEd?usp=sharing#scrollTo=UKZAgDk9Jnrx

In [18]:
!nvidia-smi

Tue Dec  7 16:32:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    38W / 250W |   1429MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#install the package
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.1 MB/s 
     |████████████████████████████████| 61 kB 609 kB/s 
     |████████████████████████████████| 596 kB 69.4 MB/s 
     |████████████████████████████████| 3.3 MB 59.8 MB/s 
     |████████████████████████████████| 895 kB 88.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
#Import sentiment model from hugging face
from transformers import pipeline
import torch.nn.functional as F
import torch
import torchvision

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
classifier = pipeline('sentiment-analysis', model = model_name, device = 0) #connecting with GPU using device variable




Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [4]:
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
import os
import time
import csv

path_in= r'/content/drive/MyDrive/MIZZOU/CAADRIA2022/Paper2/SentimentAnalysis/63861 posts from 116 locations processed/'

all_files = [f for f in listdir(path_in)]

filenames = list(filter(lambda x: x[-5:] == ('.xlsx') or x[-4:] == ('.xls'), all_files))
filenames.sort()
#print(jpg_files) 
print("1.Total Excel Files in the folder:",len(filenames))

1.Total Excel Files in the folder: 111


In [5]:
filenames[74:111]

['444_location_posts_from_603748646633262.xlsx',
 '44_location_posts_from_120255678025543.xlsx',
 '45_location_posts_from_154063217941661.xlsx',
 '476_location_posts_from_6369616934.xlsx',
 '477_location_posts_from_137211849657310.xlsx',
 '48_location_posts_from_254867661280868.xlsx',
 '52_location_posts_from_2180610162264143.xlsx',
 '53_location_posts_from_606603146129926.xlsx',
 '545_location_posts_from_309169313092.xlsx',
 '54_location_posts_from_716478915080947.xlsx',
 '5_location_posts_from_805910356085929.xlsx',
 '5_location_posts_from_929214890495345.xlsx',
 '60_location_posts_from_106109666630.xlsx',
 '61_location_posts_from_81281753994.xlsx',
 '641_location_posts_from_667667720283354.xlsx',
 '65_location_posts_from_321465621220374.xlsx',
 '666_location_posts_from_333784860295.xlsx',
 '67_location_posts_from_996512367217113.xlsx',
 '68_location_posts_from_390414797645824.xlsx',
 '692_location_posts_from_203667196339541.xlsx',
 '695_location_posts_from_35079602549.xlsx',
 '698_l

In [6]:
##**Read the data for sentiment analysis
#Read the data
FileID = 74 #change to 0
for files in filenames[74:111]: #comment this line and remove comment from the next line
#for files in filenames:
  startTime = time.time() # initiate timer
  data = pd.read_excel(path_in+filenames[FileID])
  
  #Read the raw data
  CaptionDataRaw = data.values.T[3].astype(str).tolist()
  #Clean the data and keep the first 1500 characters for keeping the tensors 
  CaptionData = [item[:500] for item in CaptionDataRaw]
  CaptionData


  ##**Sentiment analysis 
  SentimentResults = []

  #Sentiment analysis using defined model
  for list in CaptionData:
    #results = RunModel([list])
    results = classifier([list])
    for result in results:
      print(result)
      SentimentResults.append(result)
  print(SentimentResults)

  #making list of each variables of the dictionary
  label= []
  score= []
  for i in SentimentResults[0:len(SentimentResults)]:
      label.append(str(i['label']))
      score.append(str(i['score']))
  #print(label,score)

  ##**Rank count for the spreadsheet after the sentiment analysis
  Positive = label.count('POSITIVE')
  print(Positive)
  Negative = label.count('NEGATIVE')
  print(Negative)

    #Read other data for csv writeup
  Date = data.values.T[1].astype(str).tolist()
  Time = data.values.T[2].astype(str).tolist()
  User_ID = data.values.T[11].astype(str).tolist()
  LocationName = data.values.T[12].astype(str).tolist()
  Latitude = data.values.T[14].astype(str).tolist()
  Longitude = data.values.T[15].astype(str).tolist()


    #Convert the list into panda dataframe and write a backup csv file
  workspace = '/content/drive/MyDrive/MIZZOU/CAADRIA2022/Paper2/SentimentAnalysis/SentimentOutputAnalysis_PosNeg'
  csv_sentiments = os.path.join(workspace,"%s_Sentiment results_Ranking.csv"%filenames[FileID])
  list_dict = {'Date': Date,'Time': Time,'User_ID': User_ID,'LocationName': LocationName,'Longitude': Longitude,'Latitude': Latitude,'Caption': CaptionData, 'Rank': label, 'Score': score}
  df = pd.DataFrame(list_dict)
  df
  try: 
      df.to_csv(csv_sentiments, index=False,encoding='utf-8')
      print('Data is written in the folder')
  except IOError:
      print('File is not written in the folder')

  fields = [filenames[FileID], LocationName[0], Latitude[0], Longitude[0], Positive, Negative]
  with open(r'/content/drive/MyDrive/MIZZOU/CAADRIA2022/Paper2/SentimentAnalysis/SentimentOutputAnalysis_PosNeg/0_PosNeg.csv','a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)

  FileID +=1


  endTime = time.time()
  #print('Processing', filename, 'ID:', fileID,"runtime:",endTime-startTime)
print("All Done")


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


{'label': 'POSITIVE', 'score': 0.9951574206352234}
{'label': 'POSITIVE', 'score': 0.9975135326385498}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'NEGATIVE', 'score': 0.9966627955436707}
{'label': 'NEGATIVE', 'score': 0.9896642565727234}
{'label': 'NEGATIVE', 'score': 0.9031625390052795}
{'label': 'POSITIVE', 'score': 0.9970019459724426}
{'label': 'NEGATIVE', 'score': 0.9736608266830444}
{'label': 'NEGATIVE', 'score': 0.9899694323539734}
{'label': 'NEGATIVE', 'score': 0.9854947328567505}


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


Streaming output truncated to the last 5000 lines.
{'label': 'POSITIVE', 'score': 0.9988073110580444}
{'label': 'NEGATIVE', 'score': 0.9981677532196045}
{'label': 'NEGATIVE', 'score': 0.9635443091392517}
{'label': 'NEGATIVE', 'score': 0.8406571745872498}
{'label': 'POSITIVE', 'score': 0.992887556552887}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'POSITIVE', 'score': 0.9541466236114502}
{'label': 'POSITIVE', 'score': 0.8824906349182129}
{'label': 'NEGATIVE', 'score': 0.8784089684486389}
{'label': 'NEGATIVE', 'score': 0.9958205223083496}
{'label': 'NEGATIVE', 'score': 0.9893075823783875}
{'label': 'POSITIVE', 'score': 0